In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
images=np.load('data.npy')
%matplotlib inline

In [14]:
images=images[:200]
print(len(images))

200


In [28]:
import tensorflow.contrib.layers as lays
def discriminator(input,reuse=True):
    with tf.variable_scope('dis',reuse=reuse):
        net = lays.conv2d(input, 32, [5, 5], stride=2, padding='SAME',activation_fn=tf.nn.relu) #16 x16 x 32
        net = lays.conv2d(net, 16, [5, 5], stride=2, padding='SAME',activation_fn=tf.nn.relu) # 8 x 8 x 16
        net = lays.conv2d(net, 8, [5, 5], stride=4, padding='SAME',activation_fn=tf.nn.relu) #--> 2 x 2 x 8
        dim = int(np.prod(net.get_shape()[1:]))
        fc1 = tf.reshape(net, shape=[-1, dim], name='fc1')
        w2 = tf.get_variable('w2', shape=[fc1.shape[-1], 1], dtype=tf.float32,initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('b2', shape=[1], dtype=tf.float32,initializer=tf.constant_initializer(0.0))
        logits = tf.add(tf.matmul(fc1, w2), b2, name='logits')
        Dis_out = tf.nn.sigmoid(logits)
        return  Dis_out#logits

In [29]:
real_images=tf.placeholder(tf.float32, (None,32, 32, 3))
D_real=discriminator(real_images,reuse=True)
batch_size=20
h=len(images)
num_batches=h//batch_size
init=tf.global_variables_initializer()

In [30]:
samples=[] #generator examples
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_batches):
        batch_images=images[i*batch_size : (i+1)*batch_size].reshape(batch_size,32,32,3)
        Dis=sess.run(discriminator(real_images,reuse=True),feed_dict={real_images:batch_images})
        samples.append(Dis)

In [31]:
np.shape(samples)

(10, 20, 1)

In [32]:
H=np.reshape(samples,(num_batches*batch_size))

In [33]:
H[:20]

array([0.4988311 , 0.49900255, 0.49928197, 0.4997744 , 0.499452  ,
       0.49861383, 0.49962032, 0.4987932 , 0.49897295, 0.4971698 ,
       0.4999128 , 0.49775982, 0.49808574, 0.49948093, 0.49932647,
       0.49867305, 0.49797288, 0.50132024, 0.499463  , 0.50007457],
      dtype=float32)